# KaelumAI Testing

## Setup

In [1]:
cd ..

c:\Users\aadit\Projects\kaelum_mcp\kaelum


In [5]:
%pip install -e .

Note: you may need to restart the kernel to use updated packages.


c:\Users\aadit\Projects\kaelum_mcp\.venv\Scripts\python.exe: No module named pip


In [6]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.


c:\Users\aadit\Projects\kaelum_mcp\.venv\Scripts\python.exe: No module named pip


In [16]:
from kaelum import enhance, set_reasoning_model
import time

MODEL = "llama3.2:3b"

## Test LLMs

In [17]:
# Test models
models = ["llama3.2:3b", "qwen3:4b"]
query = "What is 15% of 200?"

for model in models:
    start = time.time()
    set_reasoning_model(model=model)
    result = enhance(query)
    elapsed = time.time() - start
    print(f"{model}: {elapsed:.2f}s")
    print(result[:100])
    print()

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
# Example: Using a custom local model
# Just set provider="custom" and point to your model's OpenAI-compatible endpoint

from kaelum.core.config import LLMConfig, MCPConfig
from kaelum.runtime.orchestrator import MCP

# Option 1: Ollama (already working)
config_ollama = MCPConfig(
    llm=LLMConfig(
        provider="ollama",
        model="llama3.2:3b",
    )
)

# Option 2: vLLM (if you're running vLLM server)
config_vllm = MCPConfig(
    llm=LLMConfig(
        provider="vllm",
        model="your-model-name",
        base_url="http://localhost:8000/v1",
    )
)

# Option 3: YOUR custom model (NEW!)
# Works with any OpenAI-compatible server:
# - LiteLLM, LocalAI, FastChat, llama.cpp server, etc.
config_custom = MCPConfig(
    llm=LLMConfig(
        provider="custom",
        model="your-custom-model",
        base_url="http://localhost:5000/v1",  # Your model's endpoint
        api_key="optional",  # Only if your server requires it
    )
)

# Test with Ollama (change to config_custom when you have your model running)
mcp = MCP(config_ollama)
result = mcp.infer("What is 25% of 80?")

print("Query:", result["query"])
print("\nReasoning trace:")
for i, step in enumerate(result["trace"], 1):
    print(f"{i}. {step}")
print("\nFinal answer:", result["final"])

## Test Custom Local Model

KaelumAI supports using YOUR OWN open-source model without any external API calls.
Three main options:
1. **Ollama** - easiest (supports 100+ models)
2. **vLLM** - fastest (production-grade serving)
3. **Custom provider** - most flexible (any OpenAI-compatible server)